# epiTOC2

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [30]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [31]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.epiTOC2)


class epiTOC2(pyagingModel):
    def __init__(self):
        super().__init__()
        self.delta = None
        self.beta0 = None

    def preprocess(self, x):
        """
        Replace NaNs with zero; missing features should already be imputed via reference_values.
        """
        return torch.nan_to_num(x, nan=0.0)

    def forward(self, x):
        x = self.preprocess(x)

        device = x.device
        dtype = x.dtype

        delta = self.delta.to(device=device, dtype=dtype)
        beta0 = self.beta0.to(device=device, dtype=dtype)

        denom = delta * (1 - beta0)
        denom = torch.where(denom == 0, torch.ones_like(denom), denom)

        contrib = (x - beta0) / denom
        k = contrib.size(1)
        vals = 2.0 * torch.sum(contrib, dim=1) / k

        return self.postprocess(vals.unsqueeze(1))

    def postprocess(self, x):
        return x



In [32]:
model = pya.models.epiTOC2()

## Define clock metadata

In [33]:
model.metadata["clock_name"] = 'epitoc2'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2020
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Teschendorff, Andrew E. \"A comparison of epigenetic mitotic-like clocks for cancer risk prediction.\" Genome Medicine 12.1 (2020): 56."
model.metadata["doi"] = "https://doi.org/10.1186/s13073-020-00752-3"
model.metadata["research_only"] = None
model.metadata["notes"] = "Stem cell division rate estimate using EpiTOC2."


## Download clock dependencies

#### Download coefficient file

In [34]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/EpiTOC2.csv"
os.system(f"curl -L {coeff_url} -o EpiTOC2.csv")


0

## Load features

#### From Excel file

In [35]:
df = pd.read_csv('EpiTOC2.csv', index_col=0)
df['feature'] = df.index.astype(str)
model.features = df['feature'].tolist()


## Load weights into base model


In [36]:
#### From CSV file


In [37]:
model.delta = torch.tensor(df['delta'].values, dtype=torch.float32).unsqueeze(0)
model.beta0 = torch.tensor(df['beta0'].values, dtype=torch.float32).unsqueeze(0)
model.base_model = None


In [38]:
#### Linear model


# Not used; computation happens in the model forward


In [39]:
model.reference_values = [-1]*len(model.features)

model.reference_values = [0.0] * len(model.features)

In [40]:
model.preprocess_name = "mean"
model.preprocess_dependencies = None

In [41]:
model.preprocess_name = "nan_to_zero"
model.preprocess_dependencies = None


model.postprocess_name = None
model.postprocess_dependencies = None


In [42]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Teschendorff, Andrew E. "A comparison of epigenetic mitotic-like '
             'clocks for cancer risk prediction." Genome Medicine 12.1 (2020): '
             '56.',
 'clock_name': 'epitoc2',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1186/s13073-020-00752-3',
 'notes': 'Stem cell division rate estimate using EpiTOC2.',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2020}
reference_values: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]... [Total elements: 163]
preprocess_name: 'nan_to_zero'
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg00043095', 'cg00347369', 'cg00397986', 'cg00466268', 'cg00884606', 'cg00916884', 'cg01435574', 'cg01537995'

## Basic test

In [43]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred


tensor([[ 1562.6673],
        [-3824.6137],
        [  645.1144],
        [-1810.9748],
        [ -142.2699],
        [ 1630.9714],
        [ 9229.4758],
        [ 4666.7934],
        [ 5489.9210],
        [ 1256.8648]], dtype=torch.float64)

## Save torch model

In [44]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory

In [45]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: EpiTOC2.csv
